<a href="https://colab.research.google.com/github/mr-ayush-agrawal/DL_Notebooks/blob/main/Lect/Hyperprams_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hyperparameters Tuning

We are using [Pima Indians Diabetes Dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data) for this notebook and tring to get the concept of hyperparameter tuning

A copy of same dataset has been stored in the github and using the same here.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mr-ayush-agrawal/DL_Notebooks/main/Lect/IndianDiabetes.csv')
df.shape

(768, 9)

In [4]:
df.sample(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
517,7,125,86,0,0,37.6,0.304,51,0
61,8,133,72,0,0,32.9,0.270,39,1
421,2,94,68,18,76,26.0,0.561,21,0
598,1,173,74,0,0,36.8,0.088,38,1
627,0,132,78,0,0,32.4,0.393,21,0


In [5]:
df.corr().Outcome

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [8]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x.shape, y.shape

((768, 8), (768,))

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = len(x))

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [22]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = x.shape[1]))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 32)                288       
                                                                 
Total params: 288 (1.12 KB)
Trainable params: 288 (1.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.fit(x_train, y_train, batch_size = 32, epochs = 100, validation_data = (x_test, y_test))

Epoch 1/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4106 - accuracy: 0.8094 - val_loss: 0.5341 - val_accuracy: 0.7338
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4085 - accuracy: 0.8094 - val_loss: 0.5371 - val_accuracy: 0.7208
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4079 - accuracy: 0.8046 - val_loss: 0.5389 - val_accuracy: 0.7338
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4058 - accuracy: 0.8094 - val_loss: 0.5355 - val_accuracy: 0.7403
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4048 - accuracy: 0.8094 - val_loss: 0.5380 - val_accuracy: 0.7208
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4034 - accuracy: 0.8143 - val_loss: 0.5415 - val_accuracy: 0.7143
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4021 - accuracy: 0.8176 - val_loss: 0.5476 - val_accuracy: 0.7143
Epoch 8/100
2

# Tuning

using the library called **Keras Tuner** for the purpose of Hyperparam tuning

In [33]:
# !pip install keras-tuner -U > /dev/null
!pip show keras-tuner

Name: keras-tuner
Version: 1.4.7
Summary: A Hyperparameter Tuning Library for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The KerasTuner authors
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: keras, kt-legacy, packaging, requests
Required-by: 


In [29]:
import kerastuner as kt

<ipython-input-29-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


## Tuning Optimizers

In [35]:
# here just selecting the optimizers
def build_model (hp):
    model = Sequential()
    model.add(Dense(32, activation = 'relu', input_dim = 8))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

    optimizers = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer = optimizers, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [36]:
# making the tuner object
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

In [39]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

In [41]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [42]:
model = tuner.get_best_models(num_models = 1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1377 (5.38 KB)
Trainable params: 1377 (5.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 100,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5426 - accuracy: 0.7638 - val_loss: 0.5869 - val_accuracy: 0.6948
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5017 - accuracy: 0.7704 - val_loss: 0.5679 - val_accuracy: 0.6883
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4736 - accuracy: 0.7785 - val_loss: 0.5596 - val_accuracy: 0.6688
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4601 - accuracy: 0.7769 - val_loss: 0.5528 - val_accuracy: 0.6883
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4500 - accuracy: 0.7834 - val_loss: 0.5473 - val_accuracy: 0.6883
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4441 - accuracy: 0.7834 - val_loss: 0.5429 - val_accuracy: 0.6948
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4402 - accuracy: 0.7785 - val_loss: 0.5414 - val_accuracy: 0.6948
Epoch 14

## Tuning the numeber of neurons

In [58]:
def build_model(hp):
    model = Sequential()
    units = hp.Int("Units", 8, 128)  #, step_size = 8

    model.add(Dense(units, activation='relu', input_dim=x.shape[1]))
    # model.add(Dense(units, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    return model

In [59]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 7,
                        directory = 'mydir')

In [60]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 7 Complete [00h 00m 02s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7207792401313782
Total elapsed time: 00h 00m 13s


In [61]:
tuner.get_best_hyperparameters()[0].values

{'Units': 79}

In [63]:
model = tuner.get_best_models(num_models =1)[0]

In [65]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 100,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5311 - accuracy: 0.7752 - val_loss: 0.5700 - val_accuracy: 0.7013
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5007 - accuracy: 0.7801 - val_loss: 0.5531 - val_accuracy: 0.7013
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4808 - accuracy: 0.7915 - val_loss: 0.5445 - val_accuracy: 0.7013
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4666 - accuracy: 0.7948 - val_loss: 0.5386 - val_accuracy: 0.7143
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4572 - accuracy: 0.7948 - val_loss: 0.5330 - val_accuracy: 0.7143
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4505 - accuracy: 0.7915 - val_loss: 0.5322 - val_accuracy: 0.7208
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4454 - accuracy: 0.7997 - val_loss: 0.5306 - val_accuracy: 0.7143
Epoch 14

## Number of Layers

In [66]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(79, activation = 'relu', input_dim = x.shape[1]))
    for i in range(hp.Int('Layers', min_value = 1, max_value = 10)):
        model.add(Dense(79, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    return model

In [71]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 10,
                        directory = 'mydir',
                        project_name = 'Layers')

Reloading Tuner from mydir/Layers/tuner0.json


In [72]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 01m 06s


In [73]:
tuner.get_best_hyperparameters()[0].values

{'Layers': 7}

In [74]:
model = tuner.get_best_models(num_models =1)[0]

In [76]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 100,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 3.5523e-05 - accuracy: 1.0000 - val_loss: 4.1431 - val_accuracy: 0.7143
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 3.2312e-05 - accuracy: 1.0000 - val_loss: 4.2177 - val_accuracy: 0.7143
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 2.9634e-05 - accuracy: 1.0000 - val_loss: 4.2765 - val_accuracy: 0.7143
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 2.7602e-05 - accuracy: 1.0000 - val_loss: 4.3329 - val_accuracy: 0.7143
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 2.5876e-05 - accuracy: 1.0000 - val_loss: 4.3884 - val_accuracy: 0.7143
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 2.4308e-05 - accuracy: 1.0000 - val_loss: 4.4375 - val_accuracy: 0.7143
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 2.2917e-05 - accuracy: 1.0000 - val_loss: 4.4852 - va

## Multiple Parameters

Finding the optimal value of multiple parameters